In [2]:
import pandas as pd
import numpy as np

In [2]:
path = r'GT_IPI.parquet'
df = pd.read_parquet(path,engine = 'auto')

In [3]:
import pandas as pd

def check_similarity(row1, row2, threshold=0.95):
    # Compare float values for the given rows, handling zero values
    for a, b in zip(row1, row2):
        if a == 0 or b == 0:
            if a != b:
                return False
        else:
            if min(a, b) / max(a, b) < threshold:
                return False
    return True

def process_data(df):
    results = []
    unique_indexes = []

    # Find all columns that start with "lang_pct"
    columns_to_compare = [col for col in df.columns if col.startswith("lang_pct")]

    for identifier, group in df.groupby('css_pk'):
        group = group[columns_to_compare]
        original_count = 0
        duplicate_count = 0
        unique_rows = []

        while not group.empty:
            original_row = group.iloc[0]
            original_index = group.index[0]
            duplicates = group.apply(lambda row: check_similarity(original_row, row), axis=1)
            count_duplicates = duplicates.sum() - 1  # Subtract 1 to exclude the original row
            if count_duplicates == 0:
                unique_rows.append(original_index)
            original_count += 1
            duplicate_count += count_duplicates
            group = group[~duplicates]  # Remove the duplicates and the original row

        unique_indexes.extend(unique_rows)
        results.append({'identifier': identifier, 
                        'original_count': original_count, 
                        'duplicate_count': duplicate_count,
                        'proportion_original': original_count / (original_count + duplicate_count)})

    result_df = pd.DataFrame(results)
    unique_indexes_df = pd.DataFrame(unique_indexes, columns=['unique_index'])
    return result_df, unique_indexes_df

# Example usage
result_df, unique_indexes_df = process_data(df)
result_df.to_csv('proportions_6-20.csv', index=False)
unique_indexes_df.to_csv('unique_indexes.csv', index=False)
print(result_df)
print(unique_indexes_df)


      identifier  original_count  duplicate_count  proportion_original
0        3000169             201              149             0.574286
1        3000193             128                3             0.977099
2        3000350             143              114             0.556420
3        3000374             206               31             0.869198
4        3000665             154               26             0.855556
...          ...             ...              ...                  ...
29706    5830967               5                0             1.000000
29707    5830990               1                0             1.000000
29708    5831013             157               36             0.813472
29709    5831045              17                0             1.000000
29710    5831049             221               81             0.731788

[29711 rows x 4 columns]
         unique_index
0             5289963
1             5289964
2             5289965
3             5289966
4           

In [3]:
indices = pd.read_csv('unique_indexes.csv')

In [9]:
indices

,unique_index
0,5289963
1,5289964
2,5289965
3,5289966
4,5289967
...,...
3136021,799958
3136022,799959
3136023,799960
3136024,799961


In [11]:
len(indices['unique_index'].unique())

3136026